# Lab: Deploy your ML/DL model as a scalable API in 60 minutes

Repo: https://github.com/microsoft/AIforEarth-API-Development

This document is meant to be a lab walk-through, so it contains all the instructions for you to complete the steps easily, but has minimal explanation on _how_ the technologies involved work or _why_ we use them.

For more motivation for deploying machine learning models as APIs as a way to make your work available to the community and detailed explanation of each step, see our [Quickstart](https://github.com/microsoft/AIforEarth-API-Development/blob/master/Quickstart.md#ai-for-earth---quickstart-tutorial) or the main [Readme](https://github.com/Microsoft/AIforEarth-API-Development/#ai-for-earth---creating-apis) on this repo.

## 1. Prerequisites

There are two options for working through the lab:

1. Using your laptop. Make sure to have Docker installed and working. Instructions for all operating systems can be found here: https://docs.docker.com/install/
    - Even if you’re running Windows, we use Linux containers for model deployment.  When the Docker Desktop installer asks you:
        - If you want to use Windows containers, say “no” (i.e., don’t check the box that says “use Windows containers instead of Linux containers”)
        - If you want to reboot to enable Hyper-V so you can run Linux containers, say “yes”


2. Using an Azure Data Science Virtual Machine for Linux (Ubuntu) (important to get the *Linux* version, not Windows), where Docker is already installed. Make sure you have access to an Azure Subscription and have the VM created there, and that you have good tools for editing code on the Linux VM (comfortable with emacs/vim, or use VS Code plug-ins, Sublime SFTP package, PyCharm Professional with deployment support, etc).

If you would like to deploy a model of your own during the lab instead of using a sample model, make sure to have the model file at hand and a Python or R script for loading the model and performing inference on incoming data. 

If you would like to go through the "deploy remotely" section of this tutorial, it's best to have Azure CLI [installed](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest) in your environment so you can log into an Azure Container Registry. This assumes that you have access to an Azure subscription.

## 2. Motivation

The idea is that you can use our Framework to develop and test an API locally, and the resulting container can be “dropped in” and deployed on our scalable, Kubernetes-based [platform](https://github.com/microsoft/AIforEarth-API-Platform/) with no additional changes (local libraries are swapped with distributed/scalable versions).

## 3. Preparation

### 3.1 Make sure Docker is running

In your environment (local or a Linux DSVM), make sure Docker is running:

```
docker ps
```
This lists all the Docker processes, which is probably empty right now.

If you're running into permission issues, use `sudo` with all the Docker commands. If you don’t want to preface the `docker` command with `sudo`, go through the post-installation processes detailed [here](https://docs.docker.com/install/linux/linux-postinstall/) and restart the VM.


### 3.2 Pull our example `base-py` Docker image

To save you time later on, please issue the following command to start downloading the Python-based base image:

```
docker pull mcr.microsoft.com/aiforearth/base-py:latest
```
You should see a sequence of layers get downloaded.


### 3.3 Clone this repo

Clone this repository:

```
git clone https://github.com/microsoft/AIforEarth-API-Development.git
```

In this walk-through, we will deploy a toy API using our `base-py` image, which is the most barebone Python-based Docker image in our API Framework. 

- Navigate to the `base-py` example in `AIforEarth-API-Development/Examples/base-py`.


### 3.4 Examine components of the API files

The `Examples/base-py` directory contains all the pieces of the API. To read more about what the `Dockerfile` and other files do, visit the [main readme](https://github.com/microsoft/AIforEarth-API-Development#step-2-build-an-example-service) of this repo. The most important files are:

- `Dockerfile`: describes the commands that needs to be executed on top of a base image, such as installing additional packages using `conda`, copying the API execution code (`./my_api`) to the container, setting up environment variables, and expose a port on the container (1212).
    - For the list of instructions that can be used in a Dockerfile, see https://docs.docker.com/engine/reference/builder (Docker documentation is excellent!). 

- `my_api/runserver.py`: the `my_api` folder should contain all the execution code files and model files (inference graphs, pickled model weights, etc). 
    - The script `runserver.py` is the entry point script, where the Flask app is set up and your endpoints defined. 
    
    
#### Endpoints

Notice that currently in `my_api/runserver.py`, there are two endpoints defined, marked by the `@ai4e_service.api_async_func` and the `@ai4e_service.api_sync_func` decorators. 

For a more detailed explanation of the input/output patterns, see this [section](https://github.com/microsoft/AIforEarth-API-Development/blob/master/Quickstart.md#inputoutput-patterns) in our Quickstart.

##### Async endpoint

The async/long-running endpoint `default_post` has an input validation function called `process_request_data`, defined above and referenced in the `request_processing_function` field of the decorator. It currently just reads the data passed by the user in the POST body of the call, and make it available to the `default_post` function in a keyword argument called `data`. 

This function then obtains a `taskId` in another keyword argument - this is placed here by the API Framework. You should pass this `taskId` back to the user before entering into a long-running process so they can use it at the `/task` endpoint to check the status of their request.

It then loads the serialized json body, and calls a function `run_model`, where your model inference code should take the input arguments and data, apply the model, and uploads the results somewhere or write it in the status body if the result is short (such as a classification).

##### Sync endpoint

The example synchronous endpoint is `echo`, which echos the argument the user passed in through the URL back to the user.

## 4. Deploy locally

### 4.1 Build the image
To build a Docker image from the Dockerfile in the current directory:
```
docker build . -t yasiyu.azurecr.io/my-api/1.0-example-api:1
```

The first time you build it, it will be fairly slow (even if you've previously pulled the base image) as it needs to install all the conda dependencies. Docker will cache layers that are not modified, so subsequent builds should be much faster as it is only copying the code that changed in the `my_api` folder.

Here `.` refers to the fact that we are using the current directory as the Docker build context. The `-t` argument is the tag that you can give to this image; here's a convention for naming Docker images, which is:

```
<container_registry_name>.azurecr.io/<moniker>/<image_version>-<api_name>:<build_number>
```

### 4.2 Start the container

To start a container using this image you just built:
```
docker run -p 6002:1212 yasiyu.azurecr.io/my-api/1.0-example-api:1
```
The port mapping specified using `-p` maps localhost:6002 to port 1212 in the Docker container, which you exposed in the Dockerfile. 

If you're on Windows and run into an error `standard_init_linux.go:207: exec user process caused "no such file or directory"`, see this [section](https://github.com/microsoft/AIforEarth-API-Development/blob/master/Quickstart.md#run-your-image-locally) in our Quickstart for how to fix it.


### 4.3 Test the synchronous endpoint
You can now make an API call to

```
http://localhost:6002/v1/my_api/tasker/echo/hello_world
```
to hit the sync `echo` endpoint defined in `my_api/runserver.py`. You can paste this URL string into a browser since it's a GET request; otherwise, use the following snippet to test it programmatically:

In [1]:
import requests

base_url = 'http://localhost:6002/v1/my_api/tasker/'
sync_endpoint = 'echo/'
argument = 'hello_world'

url = base_url + sync_endpoint + argument

r = requests.get(url)
print(r.text)

Echo: hello_world


### Test the asynchronous endpoint

In [2]:
import json

async_endpoint = 'example'

url = base_url + async_endpoint

payload = {'key': 'value'}
payload = json.dumps(payload)  # serialize the json payload

r = requests.post(url, data=payload)
print(r.text)

TaskId: 9365


In [3]:
task_id = r.text.split('TaskId: ')[1]

In [4]:
# check the status using the TaskID returned
r = requests.get(base_url + 'task/' + task_id)
print(r.text)

# the example async API sleeps for 10 seconds. Check status again after 10 seconds and you should
# see that the "status" is now "completed".

{"uuid": 9365, "status": "running model", "timestamp": "2019-06-07 04:44:46", "endpoint": "uri"}



### 4.4 Development process

To kill the running Docker container, open another command line window and check its container ID (first column) or name (last column) using `docker ps`, then
```
docker kill container_ID_or_name
```

You may need to repeat this build-run-kill process as you make changes and debug through your API code.


### 4.5 Using GPUs

Substitute `nvidia-docker` in the place of `docker` in the `docker build` command if you're on a GPU-enabled VM with the necessary CUDA drivers so as to use the GPU for your inference:

```
nvidia-docker build . -t yasiyu.azurecr.io/my-api/1.0-example-api:1
```


## 5. Publish the Docker image

When you are happy with how your API is working, you could push your Docker image containing all your API code and model files to Azure Container Registry (or DockerHub, if making it public). 

This allows you to pull (download) the same docker image on another VM and start a container based on that image and launch your API that way. Alternatively, you could still build the image on the VM you want to deploy to and start a container from that, if you can easily transfer the source code and model artifacts to the VM. To deploy using Azure Container Instances instead of a VM (so that you don't have to manage the VM), you would need your image pushed to an Azure Container Registry (ACR). 

Instructions for setting up your ACR are in [this section](https://github.com/microsoft/AIforEarth-API-Development/blob/master/Quickstart.md#publish-to-azure-container-registry) in our Quickstart.

Assuming you've successfully set up an ACR called `yasiyu` with login server `yasiyu.azurecr.io`, you can log in to that on the command line with the Azure CLI:

```
az acr login --name yasiyu
```

assuming you have logged in to your Azure account via `az login` and that `az account show` indicates that the default subscription is the one containing this ACR instance.


Then, push the image you built above:

```
docker push yasiyu.azurecr.io/my-api/1.0-example-api:1
```


## 6. Deploy on a VM

One way to deploy this API for people in your team or a small group of users to call is to serve it from an Azure Linux VM.

This involves starting a Docker container based on your Docker image in a [tmux session](https://hackernoon.com/a-gentle-introduction-to-tmux-8d784c404340) (or running in the background) on the VM. The tmux session allows your process to run after you've left the ssh session.

If you have pushed your image to ACR, login to your ACR on the VM, and pull the image:

```
docker pull yasiyu.azurecr.io/my-api/1.0-example-api:1
```

(It seems that you need your ACR name in all lower case...)

And start a container based on that image:

```
docker run -p 6000:1212 yasiyu.azurecr.io/my-api/1.0-example-api:1
```

Now visit Azure portal to open port 6000 on that VM. On the "Networking" section of the VM page, click on "Add inbound port rule", fill "6000" in "Destination portl ranges", and click "Add".

Your API should now be available at 
```
http://VM_DNS_name.vm_region.cloudapp.azure.com:6000
```
or
```
http://VM_IP_address:6000
```

## 7. Deploy via Azure Container Instances (ACI)

See [section](https://github.com/microsoft/AIforEarth-API-Development/blob/master/Quickstart.md#run-your-container-in-azure-container-instances) "Run your container in Azure Container Instances" in our Quickstart for how to deploy an ACI container group based on the Docker image you pushed to your ACR in Azure Portal.

Alternatively, you can deploy an ACI container group using the Azure CLI on the command line. First log in to your ACR:

```
az acr login --name yasiyu
```
Then create an ACI instance:

```
az container create --resource-group yasiyu_rg --name example-container1 --image yasiyu.azurecr.io/my-api/1.0-example-api:1 --dns-name-label yasiyu-api1 --ports 1212 --registry-username <username> --registry-password <password>
```
- You can look up the `registry-username` and `registry-password` fields in the Azure Portal page for your registry, in the "Access keys" section under "Settings".

- Note that the `ports` argument should be `1212` since that is the port we specified to expose in the Dockerfile.

- Documentation for the `az container create` command is [here](https://docs.microsoft.com/en-us/cli/azure/container?view=azure-cli-latest#az-container-create).

- The container name must contain no more than 63 characters and must match the regex `[a-z0-9]([-a-z0-9]*[a-z0-9])?` (e.g. 'my-name')

This will take a few minutes, and should print out the deployment status, with 'provisioningState' showing 'Succeeded'. Your IP address is also in this information, which you can also query using

```
az container show --resource-group yasiyu_rg --name example-container --query "{FQDN:ipAddress.fqdn}" --output table
```

Your API should now be available at this URL and the port you specified. 

In [5]:
aci_address = 'yasiyu-api1.eastus.azurecontainer.io'
aci_port = '1212'

base_url = 'http://{}:{}/v1/my_api/tasker/'.format(aci_address, aci_port)

sync_endpoint = 'echo/'
argument = 'hello_world'

url = base_url + sync_endpoint + argument
print(url)

r = requests.get(url)
print(r.text)

http://yasiyu-api1.eastus.azurecontainer.io:1212/v1/my_api/tasker/echo/hello_world
Echo: hello_world


Congratulations, you now know how to deploy a Flask-based API! 

## 8. Deploy your own model

Time to plug in your useful model! If you don't have a model that you'd like to try this with right now, we have sample code in [Examples](https://github.com/microsoft/AIforEarth-API-Development/tree/master/Examples) for PyTorch and TensorFlow (in addition, `animal-detector-api` is a model built with the TensorFlow Object Detection API) and instructions for downloading the required model files and sample data. 

You can now copy the `base-py` folder that we've built this example image with to your own repo, drop in your model file in `my-api`, and place your input handling and model inference code in `runserver.py` or another file it imports from. 

If you decide to change the name of the folder `my-api` or `runserver.py`, you also need to change the path to this entry point script in `supervisord.conf`.

## 9. Deploy on our scalable platform

The Docker image you have created in this process, once integrated with your model files and inference code, can be deployed on our hosting platform with no additional changes. The local packages for task management and telemetry will be swapped with distributed versions.

If your team would like to host the platform and make your APIs available that way, we will soon be publishing an [Azure Resource Manager](https://docs.microsoft.com/en-us/azure/azure-resource-manager/) template that describes all of our components so you could replicate easily.

## 10. Resource cleanup

Don't forget to delete all the resources that you've set up to complete this lab afterwards, most importantly any VM or Azure Container Instances (ACI) instances, but also the Azure Container Registry (you can delete individual images stored there). You can do this in the Azure Portal or through the CLI.